In [2]:
%%capture
pip install cfbd --upgrade

In [14]:
from __future__ import print_function
from cfbd.rest import ApiException
from pprint import pprint
import cfbd
import numpy as np
import pandas as pd
import time

# Walker Blackston, MSPH
# blackstonjw@icloud.com
# https://github.com/jwblackston

start = time.time()

### -----------------------------
### IMPORTING AND CLEANING DATA
### -----------------------------
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'YOUR API KEY HERE'
configuration.api_key_prefix['Authorization'] = 'Bearer'

# set up the API configs to connect to each CFB data client
api_config = cfbd.ApiClient(configuration)
recruit_api = cfbd.RecruitingApi(api_config)
draft_api = cfbd.DraftApi(api_config)
ppa_api = cfbd.MetricsApi(api_config)

# bring in recruit data - run this once and store in csv 
recruit = []

print('Bringing in recruit data...')
for year in range(2008, 2022):
    response = recruit_api.get_recruiting_players(year=year)
    recruit = [*recruit, *response]


recruit_df = pd.DataFrame().from_records([
                dict(ident = r.id, 
                     rank = r.ranking, 
                     name = r.name, 
                     year = r.year, 
                     rating=r.rating, 
                     stars=r.stars, 
                     position=r.position, 
                     commit = r.committed_to)
                for r in recruit
                if r.ranking is not None
                    and r.name is not None
                    and r.year is not None
                    and r.rating is not None
            ])

recruit_df.to_csv('recruit_df.csv', index = False)
print('Recruiting data complete.')

# bring in Draft data for same range, run once and store as csv
draft = []

print('Bringing in draft data...')
for year in range(2008, 2022):
    response = draft_api.get_draft_picks(year=year)
    draft = [*draft, *response]


draft_df = pd.DataFrame().from_records([
                dict(draft_grade = p.pre_draft_grade,
                     college = p.college_team,
                     name = p.name, 
                     year = p.year, 
                     overall=p.overall, 
                     d_round = p.round,
                     pick = p.pick,
                     position=p.position, 
                     )
                for p in draft
                if p.pick is not None
                    and p.name is not None
                    and p.year is not None
                    and p.position is not None
            ])

# label draft success = 1 by definition bc drafted
draft_df['draft_success'] = pd.Series(1 for i in range(len(draft_df.index)))

# store data locally
draft_df.to_csv('draft_df.csv', index = False)
print('Draft data complete.')

# bring in metrics, merge all metrics data and output df
ppa13 = pd.read_csv('ppa_2013.csv')
ppa14 = pd.read_csv('ppa_2014.csv')
ppa15 = pd.read_csv('ppa_2015.csv')
ppa16 = pd.read_csv('ppa_2016.csv')
ppa17 = pd.read_csv('ppa_2017.csv')
ppa18 = pd.read_csv('ppa_2018.csv')
ppa19 = pd.read_csv('ppa_2019.csv')
ppa20 = pd.read_csv('ppa_2020.csv')
ppa21 = pd.read_csv('ppa_2021.csv')

metrics_list = [ppa13, ppa14, ppa15, ppa16, ppa17, ppa18, ppa19, ppa20, ppa21]
print('Bringing in metrics data...')
metrics_df = pd.concat(metrics_list)

metrics_df.to_csv('metrics_df.csv', index = False)
print('Metrics data complete.')

# Begin merging data frames to filter down to analytic set
recruit_draft_df = pd.merge(recruit_df, draft_df,
                           how='left', left_on = ['name', 'commit'], right_on = ['name', 'college'])

dfRecruitDraftMetrics = pd.merge(recruit_draft_df, metrics_df,
                   left_on = ['name', 'college'], right_on = ['name', 'team'], how = 'left')
#label "failed" draft status 
success_def = [
    (dfRecruitDraftMetrics['d_round'] == 'NaN'),
    (dfRecruitDraftMetrics['d_round'] >= 1) & (dfRecruitDraftMetrics['d_round'] < 4),
    (dfRecruitDraftMetrics['d_round'] >= 4)
]

defs = [0, 1, 0]

dfRecruitDraftMetrics['draft_success'] = np.select(success_def, defs)

dfComplete = dfRecruitDraftMetrics[dfRecruitDraftMetrics['countablePlays'].notna()]

# make it cleaner and drop unnecessary columns
dfComplete.rename(columns = {'year_x': 'recruit_year', 'year_y':'draft_year', 
                                          'rating':'comp_recruit_rating', 'position_x':'position',
                                         'commit':'school'}, inplace = True)

dfComplete.drop(['position', 'team', 'college', 'id'], axis=1)


print('All data merged successfully.')

# store it locally and inspect:
dfComplete.to_csv('dfComplete.csv', index = False)
dfComplete.head()

print('There are ' + str(dfComplete.shape[0]) + ' recruits drafted to the NFL with PPA metrics available for analysis and modeling.')

end = time.time()
print('Time elapse (s): ', round(end - start, 1))

Bringing in recruit data...
Recruiting data complete.
Bringing in draft data...
Draft data complete.
Bringing in metrics data...
Metrics data complete.
All data merged successfully.
There are 1244 recruits drafted to the NFL with PPA metrics available for analysis and modeling.
Time elapse (s):  2473.0
